In [19]:
import pandas as pd
from jiwer import cer, wer
from tqdm import tqdm
import numpy as np

In [20]:
whisper_preds_path='/Users/markjos/projects/malachor5/data/pyarrow-datasets/tira-clean-split/tira-val-whisper.csv'
df=pd.read_csv(whisper_preds_path, keep_default_na=False, index_col='path').sort_index()
df.head()

,english,german,spanish,russian,korean,french,japanese,portuguese,turkish,polish,...,sinhala,albanian,turkmen,tatar,yiddish,latin,nynorsk,sanskrit,hawaiian,sundanese
path,,,,,,,,,,,,,,,,,,,,,
HH01082021-m01s25ms035-m01s26ms975.wav,and I'll catch you later.,und ich habe das,y yo que chelo,И я качал,그리고...,et je vais qu'est-ce que l'on,そして、,E eu que chego,ve o da,i do... Kecielo,...,අපි අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Ia ke qe lo,Ia ke chelo,И јо кешело,"יא, קצ'לו.",y yoha ke chelo,ლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლ...,"y yoh, ke che lo",y yaw ke chelo,Iwa ke chelo
HH01082021-m01s42ms440-m01s43ms330.wav,Kitchelow,Kichelow,que chaval,Качало!,끝이 없네,Qu'est ce que je veux?,ケチャロン,Que chalo!,Kötü çalar,Kieczelon,...,අපි කින් කින් කින් කින් කින් කින් කින් කින් ක...,Ke qe ljoh,qe chalo,Кетчало!,קצ' שלו,kê chalo?,ស្្្្្,kê chalo,ស្្្្្្្្្,Ketjalo
HH01082021-m01s45ms710-m01s46ms770.wav,Ketchup,Ketchup,que echalo,Кэчело,그쵸?,qu'est ce que je l'ai,きっちょの,quecham-o,Kötü,Keczalo,...,අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Ke qe l'o,qe chelo,кетчало,קצ'לו,ស្្្្្្្្្,ស្្្្,វ្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្...,ស្្្្្្្្,ke chelo
HH01082021-m01s59ms245-m02s01ms005.wav,and what's up?,Und jetzt geht es los!,y yo que chelo,"И о, качалу!","이어, 그 체로","Et au-delà de ce que je l'ai fait,",いいよー!けっちょろ!,E ó que chalo,Yaa keçelo,"I jo, kechalo.",...,අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Ijo ke qe qe lo,"Iyo, ke chelo.",И јо кетчало.,"יא, קצ'לו.",yoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoy...,ḍᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃ...,y y y y y y y y y y y y y y y y y y y y y y y...,yoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoy...,Ia ke chelo
HH01082021-m02s32ms780-m02s34ms960.wav,"After that, this is the end.",dann die Kälte,apre y hechelo,"А после, это все.",그리고 이 부분을,"après, il y a le chelon",アプリ、イエチロ,apre e echelo,sonra bu çelik,a potem je czelowe,...,අපි අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Apre je qe lo,apre yechelo,апре ечело.,הפרי יצלו,apre ye chelo,"after that, it's done",apre ye chelo,apre ye chelo,apre ye chelo


In [21]:
df['italian']

path
HH01082021-m01s25ms035-m01s26ms975.wav     e io che ce l'ho
HH01082021-m01s42ms440-m01s43ms330.wav        che c'è l'ho?
HH01082021-m01s45ms710-m01s46ms770.wav        che è cell'uo
HH01082021-m01s59ms245-m02s01ms005.wav      io che c'è l'ho
HH01082021-m02s32ms780-m02s34ms960.wav       apre e ecchelo
                                                ...        
HH20240223-m17s47ms620-m17s48ms710.wav                    e
HH20240223-m17s58ms410-m17s59ms460.wav                    e
HH20240223-m25s27ms213-m25s27ms958.wav         Alla eredio.
HH20240223-m25s30ms255-m25s31ms075.wav          la l'erredo
HH20240223-m26s10ms220-m26s11ms365.wav       alla la re edo
Name: italian, Length: 2048, dtype: object

In [22]:
tira_csv_path='/Users/markjos/projects/malachor5/data/tira-clean/metadata.csv'
tira_df=pd.read_csv(tira_csv_path, index_col='path').sort_index()
tira_df.head()

,Unnamed: 0,start,end,transcription,eaf_source,wav_source,raw_transcription,clip,wav_rawpath,file_name,...,trans_len_sq,trans_len_log,cos_sim_softmax,cos_sim_log,duration,split,croatian,croatian_nmzd,tira_transliterated,tira_transliterated_ids
path,,,,,,,,,,,,,,,,,,,,,
HH01082021-m01s25ms035-m01s26ms975.wav,8210,85035.0,86975.0,ìjɔ̀ kə̀cə̀lò,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,ìjɔ̀ kə̀tʃə̀lò,data/tira-asr/clips/HH01082021-m01s25ms035-m01...,/Users/markjos/Library/CloudStorage/GoogleDriv...,clips/HH01082021-m01s25ms035-m01s26ms975.wav,...,2.708050,2.708050,0.000044,-10.029001,1.94,validation,i ja ću će,i ja ću će,� hocheculo,"[50258, 50291, 50360, 50364, 239, 1106, 1876, ..."
HH01082021-m01s42ms440-m01s43ms330.wav,11317,102440.0,103330.0,kə̀cə̀lò,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,kə̀tʃə̀lò,data/tira-asr/clips/HH01082021-m01s42ms440-m01...,/Users/markjos/Library/CloudStorage/GoogleDriv...,clips/HH01082021-m01s42ms440-m01s43ms330.wav,...,2.197225,2.197225,0.000044,-10.041498,0.89,validation,krećemo,krećemo,checulo,"[50258, 50291, 50360, 50364, 1876, 25436, 50257]"
HH01082021-m01s45ms710-m01s46ms770.wav,11229,105710.0,106770.0,kə̀cə̀lò,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,kə̀tʃə̀lò,data/tira-asr/clips/HH01082021-m01s45ms710-m01...,/Users/markjos/Library/CloudStorage/GoogleDriv...,clips/HH01082021-m01s45ms710-m01s46ms770.wav,...,2.197225,2.197225,0.000044,-10.041116,1.06,validation,krećemo,krećemo,checulo,"[50258, 50291, 50360, 50364, 1876, 25436, 50257]"
HH01082021-m01s59ms245-m02s01ms005.wav,5569,119245.0,121005.0,ìjɔ̀ kə̀cə̀lò,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,ìjɔ̀ kə̀tʃə̀lò,data/tira-asr/clips/HH01082021-m01s59ms245-m02...,/Users/markjos/Library/CloudStorage/GoogleDriv...,clips/HH01082021-m01s59ms245-m02s01ms005.wav,...,2.708050,2.708050,0.000045,-10.019013,1.76,validation,i joj ukećemo,i joj ukećemo,� hocheculo,"[50258, 50291, 50360, 50364, 239, 1106, 1876, ..."
HH01082021-m02s27ms480-m02s28ms550.wav,2709,147480.0,148550.0,àprì,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprì,data/tira-asr/clips/HH01082021-m02s27ms480-m02...,/Users/markjos/Library/CloudStorage/GoogleDriv...,clips/HH01082021-m02s27ms480-m02s28ms550.wav,...,1.791759,1.791759,0.000045,-10.005821,1.07,test,apri,apri,apr i,"[50258, 50291, 50360, 50364, 257, 1424, 741, 5..."


In [23]:
tira_df=tira_df[tira_df['split']=='validation']
df.shape, tira_df.shape

((2048, 99), (2048, 32))

In [24]:
df['tira']=tira_df['transcription'].to_list()
df['tira']

path
HH01082021-m01s25ms035-m01s26ms975.wav      ìjɔ̀ kə̀cə̀lò
HH01082021-m01s42ms440-m01s43ms330.wav            kə̀cə̀lò
HH01082021-m01s45ms710-m01s46ms770.wav            kə̀cə̀lò
HH01082021-m01s59ms245-m02s01ms005.wav      ìjɔ̀ kə̀cə̀lò
HH01082021-m02s32ms780-m02s34ms960.wav     àprí jə̀cə̀lò
                                                ...        
HH20240223-m17s47ms620-m17s48ms710.wav       ðál ðô ìdú
HH20240223-m17s58ms410-m17s59ms460.wav       ðál ðô ìdú
HH20240223-m25s27ms213-m25s27ms958.wav    lál lɛ́ré ídú
HH20240223-m25s30ms255-m25s31ms075.wav    lál lɛ́ré ídú
HH20240223-m26s10ms220-m26s11ms365.wav    lál lɛ́ré ídú
Name: tira, Length: 2048, dtype: object

In [25]:
yoruba_cer=cer(df['tira'].tolist(), df['yoruba'].tolist())
yoruba_cer

5.523120106367262

In [26]:
(df['tira'].str.strip()=='').value_counts()

tira
False    2048
Name: count, dtype: int64

In [27]:
metrics={}
for column in tqdm(df.columns):
    if column=='tira' or column=='path':
        continue
    # avoid errors from empty strings
    df[column]=df[column].apply(lambda s: 'q' if s.strip()=='' else s)
    col_cer=cer(df['tira'].tolist(), df[column].tolist())
    col_wer=wer(df['tira'].tolist(), df[column].tolist())
    metrics[column]={'wer': col_wer, 'cer': col_cer}
metrics

100%|██████████| 100/100 [00:21<00:00,  4.69it/s]


{'english': {'wer': 2.2033188065705667, 'cer': 1.2451301099550471},
 'german': {'wer': 2.15135769359705, 'cer': 1.3352468184792015},
 'spanish': {'wer': 2.7018102581294, 'cer': 1.4371187978810966},
 'russian': {'wer': 1.8027153871941, 'cer': 1.3555705632821897},
 'korean': {'wer': 3.71656050955414, 'cer': 2.143194816706414},
 'french': {'wer': 2.8809922896413007, 'cer': 1.6664204461515733},
 'japanese': {'wer': 1.0509554140127388, 'cer': 1.6655551569128169},
 'portuguese': {'wer': 5.3697619845792826, 'cer': 2.276027267163329},
 'turkish': {'wer': 2.163090848139457, 'cer': 1.3447016862587848},
 'polish': {'wer': 2.154877639959772, 'cer': 1.2094633096258152},
 'catalan': {'wer': 5.544753603754609, 'cer': 2.7039233480362155},
 'dutch': {'wer': 2.162923231645994, 'cer': 1.2691893717155942},
 'arabic': {'wer': 2.9210526315789473, 'cer': 1.9232636177532025},
 'swedish': {'wer': 2.139792155548106, 'cer': 1.243779414557964},
 'italian': {'wer': 2.9664767013074087, 'cer': 1.5648650359833696},
 

In [28]:
min_lang=min(metrics, key=lambda k: metrics[k]['cer'])
min_lang, metrics[min_lang]


('croatian', {'wer': 1.4627891384512235, 'cer': 0.7631217947365089})

In [29]:
df['croatian']

path
HH01082021-m01s25ms035-m01s26ms975.wav        Iga, kečelo.
HH01082021-m01s42ms440-m01s43ms330.wav           Kaj čelo?
HH01082021-m01s45ms710-m01s46ms770.wav           Kaj čelo?
HH01082021-m01s59ms245-m02s01ms005.wav     I joj, ke čelo?
HH01082021-m02s32ms780-m02s34ms960.wav      A pre je čelo.
                                                ...       
HH20240223-m17s47ms620-m17s48ms710.wav        da ldo i ru.
HH20240223-m17s58ms410-m17s59ms460.wav      zalvo i dobro.
HH20240223-m25s27ms213-m25s27ms958.wav         Galle Redu.
HH20240223-m25s30ms255-m25s31ms075.wav         Nalle Redu.
HH20240223-m26s10ms220-m26s11ms365.wav     Zala le re edu.
Name: croatian, Length: 2048, dtype: object

In [30]:
metrics_df=pd.DataFrame(metrics).transpose()
metrics_df.head()

,wer,cer
english,2.203319,1.245130
german,2.151358,1.335247
spanish,2.701810,1.437119
russian,1.802715,1.355571
korean,3.716561,2.143195


In [34]:
metrics_df.sort_values('cer')[:20]

,wer,cer
croatian,1.462789,0.763122
sundanese,1.349480,0.770276
slovenian,1.485753,0.784205
bosnian,1.497318,0.791719
serbian,1.557995,0.834244
finnish,1.366410,0.852479
lithuanian,1.440329,0.856932
maori,1.624874,0.866598
malagasy,1.434294,0.877213
estonian,1.540228,0.913281


In [37]:
metrics_df.sort_values('cer', ascending=False)[:20]

,wer,cer
georgian,21.989775,9.009687
sinhala,5.251592,7.586202
shona,13.158062,6.545385
mongolian,8.447368,6.131608
khmer,15.548609,6.015554
nynorsk,3.859873,5.803010
yoruba,10.684211,5.523120
breton,8.017097,5.513876
kannada,1.775394,5.491738
telugu,4.535535,4.991854


In [32]:
metrics_df['cer'].mean()

2.1717988188663324

In [39]:
metrics_df.loc[['yoruba', 'swahili']]

,wer,cer
yoruba,10.684211,5.52312
swahili,3.825846,1.97421
